# 

In [27]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"
#r "nuget: XPlot.Plotly.Interactive"

In [22]:


using System;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using System.Collections.Generic;
using XPlot.Plotly;

// Définir les schémas des données d'entrée et de sortie
public class SalesData
{
    public float Age { get; set; }
    public float Income { get; set; }
    public float FamilyStatus { get; set; } // ex: 0 = Single, 1 = Married, etc.
    public float EducationLevel { get; set; } // ex: 0 = High School, 1 = Bachelor, 2 = Master, etc.
    public float ContractType { get; set; } // ex: 0 = Health, 1 = Disability, 2 = Life
    public float ContractDuration { get; set; }
    public float PremiumAmount { get; set; }
    public float Region { get; set; } // ex: 0 = Urban, 1 = Rural
    public float SalesAmount { get; set; }
}

public class SalesPrediction
{
    [ColumnName("Score")]
    public float SalesAmount { get; set; }
}

// Créer et préparer les données d'entraînement
var mlContext = new MLContext();

// Exemple de données d'entraînement (à remplacer par vos propres données)
var trainingData = new List<SalesData>
{
    new SalesData { Age = 30, Income = 50000, FamilyStatus = 1, EducationLevel = 1, ContractType = 0, ContractDuration = 5, PremiumAmount = 300, Region = 0, SalesAmount = 10000 }, 
    new SalesData { Age = 45, Income = 75000, FamilyStatus = 1, EducationLevel = 2, ContractType = 1, ContractDuration = 10, PremiumAmount = 500, Region = 0, SalesAmount = 15000 },
    new SalesData { Age = 50, Income = 100000, FamilyStatus = 0, EducationLevel = 2, ContractType = 2, ContractDuration = 15, PremiumAmount = 700, Region = 1, SalesAmount = 20000 },
    new SalesData { Age = 30, Income = 55000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 5, PremiumAmount = 350, Region = 1, SalesAmount = 12000 },
    new SalesData { Age = 40, Income = 60000, FamilyStatus = 1, EducationLevel = 0, ContractType = 0, ContractDuration = 8, PremiumAmount = 400, Region = 0, SalesAmount = 14000 },
    new SalesData { Age = 35, Income = 65000, FamilyStatus = 0, EducationLevel = 2, ContractType = 2, ContractDuration = 7, PremiumAmount = 450, Region = 1, SalesAmount = 16000 },
    new SalesData { Age = 55, Income = 80000, FamilyStatus = 1, EducationLevel = 1, ContractType = 0, ContractDuration = 10, PremiumAmount = 600, Region = 1, SalesAmount = 18000 },
    new SalesData { Age = 25, Income = 40000, FamilyStatus = 0, EducationLevel = 0, ContractType = 1, ContractDuration = 3, PremiumAmount = 250, Region = 0, SalesAmount = 8000 },
    new SalesData { Age = 65, Income = 90000, FamilyStatus = 1, EducationLevel = 2, ContractType = 2, ContractDuration = 12, PremiumAmount = 750, Region = 0, SalesAmount = 22000 },
    new SalesData { Age = 50, Income = 70000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 15, PremiumAmount = 500, Region = 1, SalesAmount = 13000 },
    new SalesData { Age = 38, Income = 72000, FamilyStatus = 1, EducationLevel = 2, ContractType = 0, ContractDuration = 9, PremiumAmount = 520, Region = 1, SalesAmount = 17000 },
    new SalesData { Age = 44, Income = 68000, FamilyStatus = 1, EducationLevel = 1, ContractType = 2, ContractDuration = 11, PremiumAmount = 570, Region = 0, SalesAmount = 19000 },
    new SalesData { Age = 60, Income = 85000, FamilyStatus = 1, EducationLevel = 2, ContractType = 1, ContractDuration = 13, PremiumAmount = 650, Region = 0, SalesAmount = 21000 },
    new SalesData { Age = 27, Income = 44000, FamilyStatus = 0, EducationLevel = 0, ContractType = 0, ContractDuration = 4, PremiumAmount = 300, Region = 1, SalesAmount = 9000 },
    new SalesData { Age = 31, Income = 47000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 6, PremiumAmount = 320, Region = 0, SalesAmount = 11000 },
};

// Charger les données d'entraînement
var trainingDataView = mlContext.Data.LoadFromEnumerable(trainingData);

// Définir la pipeline d'apprentissage automatique
var pipeline = mlContext.Transforms.Concatenate("Features", 
                                                 "Age", 
                                                 "Income", 
                                                 "FamilyStatus", 
                                                 "EducationLevel", 
                                                 "ContractType", 
                                                 "ContractDuration", 
                                                 "PremiumAmount", 
                                                 "Region")
    //.Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "SalesAmount", maximumNumberOfIterations: 100));
    .Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "SalesAmount", maximumNumberOfIterations: 100));

// Entraîner le modèle
var model = pipeline.Fit(trainingDataView);

// Évaluer le modèle
var testData = trainingDataView;  // Utilisation des mêmes données pour simplifier l'exemple
var predictions = model.Transform(testData);
var metrics = mlContext.Regression.Evaluate(predictions, labelColumnName: "SalesAmount");

Console.WriteLine($"R-squared: {metrics.RSquared}");
Console.WriteLine($"Mean Absolute Error: {metrics.MeanAbsoluteError}");

// Faire des prédictions
var predictionFunc = mlContext.Model.CreatePredictionEngine<SalesData, SalesPrediction>(model);

var newSalesData = new SalesData { Age = 35, Income = 60000, FamilyStatus = 1, EducationLevel = 1, ContractType = 0, ContractDuration = 5, PremiumAmount = 400, Region = 0 };
var prediction = predictionFunc.Predict(newSalesData);

Console.WriteLine($"Predicted Sales Amount: {prediction.SalesAmount}");

// Collecte des données réelles et prédites pour le graphique
var actualSales = trainingData.Select(x => x.SalesAmount).ToArray();
var predictedSales = mlContext.Data.CreateEnumerable<SalesPrediction>(predictions, reuseRowObject: false).Select(x => x.SalesAmount).ToArray();
var ages = trainingData.Select(x => (double)x.Age).ToArray();
var incomes = trainingData.Select(x => (double)x.Income).ToArray();

// Tracer les données réelles en 3D
var trace1 = new Scatter3d
{
    x = ages,
    y = incomes,
    z = actualSales,
    mode = "markers",
    name = "Actual Sales"
};

// Tracer les données prédites en 3D
var trace2 = new Scatter3d
{
    x = ages,
    y = incomes,
    z = predictedSales,
    mode = "markers",
    name = "Predicted Sales"
};

var layout = new Layout.Layout
{
    title = "Comparaison des ventes réelles et prédites en 3D",
    scene = new Scene
    {
        xaxis = new Xaxis
        {
            title = "Âge"
        },
        yaxis = new Yaxis
        {
            title = "Revenu"
        },
        zaxis = new Zaxis
        {
            title = "Montant des Ventes"
        }
    }
};

var chart = Chart.Plot(new[] { trace1, trace2 }, layout);
display(chart);


In [24]:
using System;
using Microsoft.ML;
using Microsoft.ML.Data;
using System.Collections.Generic;
using System.Linq;
using XPlot.Plotly;

// Définir les schémas des données d'entrée et de sortie
public class SalesData
{
    public float Age { get; set; }
    public float Income { get; set; }
    public float FamilyStatus { get; set; } // ex: 0 = Single, 1 = Married, etc.
    public float EducationLevel { get; set; } // ex: 0 = High School, 1 = Bachelor, 2 = Master, etc.
    public float ContractType { get; set; } // ex: 0 = Health, 1 = Disability, 2 = Life
    public float ContractDuration { get; set; }
    public float PremiumAmount { get; set; }
    public float Region { get; set; } // ex: 0 = Urban, 1 = Rural
    public float SalesAmount { get; set; }
}

public class SalesPrediction
{
    [ColumnName("Score")]
    public float SalesAmount { get; set; }
}

var mlContext = new MLContext();

// Exemple de données d'entraînement (à remplacer par vos propres données)
var trainingData = new List<SalesData>
{
    new SalesData { Age = 30, Income = 50000, FamilyStatus = 1, EducationLevel = 1, ContractType = 0, ContractDuration = 5, PremiumAmount = 300, Region = 0, SalesAmount = 10000 },
    new SalesData { Age = 45, Income = 75000, FamilyStatus = 1, EducationLevel = 2, ContractType = 1, ContractDuration = 10, PremiumAmount = 500, Region = 0, SalesAmount = 15000 },
    new SalesData { Age = 50, Income = 100000, FamilyStatus = 0, EducationLevel = 2, ContractType = 2, ContractDuration = 15, PremiumAmount = 700, Region = 1, SalesAmount = 20000 },
    new SalesData { Age = 30, Income = 55000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 5, PremiumAmount = 350, Region = 1, SalesAmount = 12000 },
    new SalesData { Age = 40, Income = 60000, FamilyStatus = 1, EducationLevel = 0, ContractType = 0, ContractDuration = 8, PremiumAmount = 400, Region = 0, SalesAmount = 14000 },
    new SalesData { Age = 35, Income = 65000, FamilyStatus = 0, EducationLevel = 2, ContractType = 2, ContractDuration = 7, PremiumAmount = 450, Region = 1, SalesAmount = 16000 },
    new SalesData { Age = 55, Income = 80000, FamilyStatus = 1, EducationLevel = 1, ContractType = 0, ContractDuration = 10, PremiumAmount = 600, Region = 1, SalesAmount = 18000 },
    new SalesData { Age = 25, Income = 40000, FamilyStatus = 0, EducationLevel = 0, ContractType = 1, ContractDuration = 3, PremiumAmount = 250, Region = 0, SalesAmount = 8000 },
    new SalesData { Age = 65, Income = 90000, FamilyStatus = 1, EducationLevel = 2, ContractType = 2, ContractDuration = 12, PremiumAmount = 750, Region = 0, SalesAmount = 22000 },
    new SalesData { Age = 50, Income = 70000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 15, PremiumAmount = 500, Region = 1, SalesAmount = 13000 },
    new SalesData { Age = 38, Income = 72000, FamilyStatus = 1, EducationLevel = 2, ContractType = 0, ContractDuration = 9, PremiumAmount = 520, Region = 1, SalesAmount = 17000 },
    new SalesData { Age = 44, Income = 68000, FamilyStatus = 1, EducationLevel = 1, ContractType = 2, ContractDuration = 11, PremiumAmount = 570, Region = 0, SalesAmount = 19000 },
    new SalesData { Age = 60, Income = 85000, FamilyStatus = 1, EducationLevel = 2, ContractType = 1, ContractDuration = 13, PremiumAmount = 650, Region = 0, SalesAmount = 21000 },
    new SalesData { Age = 27, Income = 44000, FamilyStatus = 0, EducationLevel = 0, ContractType = 0, ContractDuration = 4, PremiumAmount = 300, Region = 1, SalesAmount = 9000 },
    new SalesData { Age = 31, Income = 47000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 6, PremiumAmount = 320, Region = 0, SalesAmount = 11000 },
};

// Charger les données d'entraînement
var fullDataView = mlContext.Data.LoadFromEnumerable(trainingData);

// Séparer les données entre entraînement et test (80% entraînement, 20% test)
var trainTestData = mlContext.Data.TrainTestSplit(fullDataView, testFraction: 0.2);
var trainDataView = trainTestData.TrainSet;
var testDataView = trainTestData.TestSet;

// Définir la pipeline d'apprentissage automatique
var pipeline = mlContext.Transforms.Categorical.OneHotEncoding(new[]
    {
        new InputOutputColumnPair("FamilyStatus"),
        new InputOutputColumnPair("EducationLevel"),
        new InputOutputColumnPair("ContractType"),
        new InputOutputColumnPair("Region")
    })
    .Append(mlContext.Transforms.Concatenate("Features", 
                                             "Age", 
                                             "Income", 
                                             "FamilyStatus", 
                                             "EducationLevel", 
                                             "ContractType", 
                                             "ContractDuration", 
                                             "PremiumAmount", 
                                             "Region"))
    .Append(mlContext.Transforms.NormalizeMinMax("Features"))
    .Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "SalesAmount", maximumNumberOfIterations: 100));

// Entraîner le modèle
var model = pipeline.Fit(trainDataView);

// Évaluer les performances du modèle sur les données d'entraînement
var trainPredictions = model.Transform(trainDataView);
var trainMetrics = mlContext.Regression.Evaluate(trainPredictions, labelColumnName: "SalesAmount");
Console.WriteLine("Performance sur le jeu d'entraînement :");
Console.WriteLine($"  R-squared : {trainMetrics.RSquared}");
Console.WriteLine($"  Mean Absolute Error : {trainMetrics.MeanAbsoluteError}");

// Évaluer les performances du modèle sur les données de test
var testPredictions = model.Transform(testDataView);
var testMetrics = mlContext.Regression.Evaluate(testPredictions, labelColumnName: "SalesAmount");
Console.WriteLine("Performance sur le jeu de test :");
Console.WriteLine($"  R-squared : {testMetrics.RSquared}");
Console.WriteLine($"  Mean Absolute Error : {testMetrics.MeanAbsoluteError}");

// Faire des prédictions
var predictionFunc = mlContext.Model.CreatePredictionEngine<SalesData, SalesPrediction>(model);

var newSalesData = new SalesData { Age = 35, Income = 60000, FamilyStatus = 1, EducationLevel = 1, ContractType = 0, ContractDuration = 5, PremiumAmount = 400, Region = 0 };
var prediction = predictionFunc.Predict(newSalesData);

Console.WriteLine($"Predicted Sales Amount: {prediction.SalesAmount}");

// Collecte des données réelles et prédites pour le graphique
var trainActualSales = mlContext.Data.CreateEnumerable<SalesData>(trainDataView, reuseRowObject: false).Select(x => x.SalesAmount).ToArray();
var trainPredictedSales = mlContext.Data.CreateEnumerable<SalesPrediction>(trainPredictions, reuseRowObject: false).Select(x => x.SalesAmount).ToArray();
var trainAges = mlContext.Data.CreateEnumerable<SalesData>(trainDataView, reuseRowObject: false).Select(x => (double)x.Age).ToArray();
var trainIncomes = mlContext.Data.CreateEnumerable<SalesData>(trainDataView, reuseRowObject: false).Select(x => (double)x.Income).ToArray();

var testActualSales = mlContext.Data.CreateEnumerable<SalesData>(testDataView, reuseRowObject: false).Select(x => x.SalesAmount).ToArray();
var testPredictedSales = mlContext.Data.CreateEnumerable<SalesPrediction>(testPredictions, reuseRowObject: false).Select(x => x.SalesAmount).ToArray();
var testAges = mlContext.Data.CreateEnumerable<SalesData>(testDataView, reuseRowObject: false).Select(x => (double)x.Age).ToArray();
var testIncomes = mlContext.Data.CreateEnumerable<SalesData>(testDataView, reuseRowObject: false).Select(x => (double)x.Income).ToArray();

// Tracer les données d'entraînement réelles et prédites en 3D
var trainTrace1 = new Scatter3d
{
    x = trainAges,
    y = trainIncomes,
    z = trainActualSales,
    mode = "markers",
    name = "Train Actual Sales"
};

var trainTrace2 = new Scatter3d
{
    x = trainAges,
    y = trainIncomes,
    z = trainPredictedSales,
    mode = "markers",
    name = "Train Predicted Sales"
};

// Tracer les données de test réelles et prédites en 3D
var testTrace1 = new Scatter3d
{
    x = testAges,
    y = testIncomes,
    z = testActualSales,
    mode = "markers",
    name = "Test Actual Sales"
};

var testTrace2 = new Scatter3d
{
    x = testAges,
    y = testIncomes,
    z = testPredictedSales,
    mode = "markers",
    name = "Test Predicted Sales"
};

var layout = new Layout.Layout
{
    title = "Comparaison des ventes réelles et prédites en 3D",
    scene = new Scene
    {
        xaxis = new Xaxis
        {
            title = "Âge"
        },
        yaxis = new Yaxis
        {
            title = "Revenu"
        },
        zaxis = new Zaxis
        {
            title = "Montant des Ventes"
        }
    }
};

var chart = Chart.Plot(new[] { trainTrace1, trainTrace2, testTrace1, testTrace2 }, layout);
display(chart);


In [47]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Models.Attributes;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using XPlot.Plotly;
using Range = Microsoft.ML.Probabilistic.Models.Range;

// Définir les schémas des données d'entrée et de sortie
public class SalesData
{
    public float Age { get; set; }
    public float Income { get; set; }
    public float FamilyStatus { get; set; }
    public float EducationLevel { get; set; }
    public float ContractType { get; set; }
    public float ContractDuration { get; set; }
    public float PremiumAmount { get; set; }
    public float Region { get; set; }
    public float SalesAmount { get; set; }
}

// Définir la classe des features transformées.
public class TransformedSalesData
{
    [VectorType]
    public float[] Features { get; set; }
    public float SalesAmount { get; set; }
}

public class SalesPrediction
{
    public float SalesAmount { get; set; }
}

var mlContext = new MLContext();

// Exemple de données d'entraînement (à remplacer par vos propres données)
var trainingData = new List<SalesData>
{
    new SalesData { Age = 30, Income = 50000, FamilyStatus = 1, EducationLevel = 1, ContractType = 0, ContractDuration = 5, PremiumAmount = 300, Region = 0, SalesAmount = 10000 },
    new SalesData { Age = 45, Income = 75000, FamilyStatus = 1, EducationLevel = 2, ContractType = 1, ContractDuration = 10, PremiumAmount = 500, Region = 0, SalesAmount = 15000 },
    new SalesData { Age = 50, Income = 100000, FamilyStatus = 0, EducationLevel = 2, ContractType = 2, ContractDuration = 15, PremiumAmount = 700, Region = 1, SalesAmount = 20000 },
    new SalesData { Age = 30, Income = 55000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 5, PremiumAmount = 350, Region = 1, SalesAmount = 12000 },
    new SalesData { Age = 40, Income = 60000, FamilyStatus = 1, EducationLevel = 0, ContractType = 0, ContractDuration = 8, PremiumAmount = 400, Region = 0, SalesAmount = 14000 },
    new SalesData { Age = 35, Income = 65000, FamilyStatus = 0, EducationLevel = 2, ContractType = 2, ContractDuration = 7, PremiumAmount = 450, Region = 1, SalesAmount = 16000 },
    new SalesData { Age = 55, Income = 80000, FamilyStatus = 1, EducationLevel = 1, ContractType = 0, ContractDuration = 10, PremiumAmount = 600, Region = 1, SalesAmount = 18000 },
    new SalesData { Age = 25, Income = 40000, FamilyStatus = 0, EducationLevel = 0, ContractType = 1, ContractDuration = 3, PremiumAmount = 250, Region = 0, SalesAmount = 8000 },
    new SalesData { Age = 65, Income = 90000, FamilyStatus = 1, EducationLevel = 2, ContractType = 2, ContractDuration = 12, PremiumAmount = 750, Region = 0, SalesAmount = 22000 },
    new SalesData { Age = 50, Income = 70000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 15, PremiumAmount = 500, Region = 1, SalesAmount = 13000 },
    new SalesData { Age = 38, Income = 72000, FamilyStatus = 1, EducationLevel = 2, ContractType = 0, ContractDuration = 9, PremiumAmount = 520, Region = 1, SalesAmount = 17000 },
    new SalesData { Age = 44, Income = 68000, FamilyStatus = 1, EducationLevel = 1, ContractType = 2, ContractDuration = 11, PremiumAmount = 570, Region = 0, SalesAmount = 19000 },
    new SalesData { Age = 60, Income = 85000, FamilyStatus = 1, EducationLevel = 2, ContractType = 1, ContractDuration = 13, PremiumAmount = 650, Region = 0, SalesAmount = 21000 },
    new SalesData { Age = 27, Income = 44000, FamilyStatus = 0, EducationLevel = 0, ContractType = 0, ContractDuration = 4, PremiumAmount = 300, Region = 1, SalesAmount = 9000 },
    new SalesData { Age = 31, Income = 47000, FamilyStatus = 0, EducationLevel = 1, ContractType = 1, ContractDuration = 6, PremiumAmount = 320, Region = 0, SalesAmount = 11000 },
};

// Charger les données d'entraînement
var fullDataView = mlContext.Data.LoadFromEnumerable(trainingData);

// Séparer les données entre entraînement et test (80% entraînement, 20% test)
var trainTestData = mlContext.Data.TrainTestSplit(fullDataView, testFraction: 0.2);
var trainDataView = trainTestData.TrainSet;
var testDataView = trainTestData.TestSet;

// Définir la pipeline d'apprentissage automatique
var pipeline = mlContext.Transforms.Categorical.OneHotEncoding(new[]
    {
        new InputOutputColumnPair("FamilyStatus"),
        new InputOutputColumnPair("EducationLevel"),
        new InputOutputColumnPair("ContractType"),
        new InputOutputColumnPair("Region")
    })
    .Append(mlContext.Transforms.Concatenate("Features", 
                                             "Age", 
                                             "Income", 
                                             "FamilyStatus", 
                                             "EducationLevel", 
                                             "ContractType", 
                                             "ContractDuration", 
                                             "PremiumAmount", 
                                             "Region"))
    .Append(mlContext.Transforms.NormalizeMinMax("Features"))
    .Append(mlContext.Transforms.CopyColumns("SalesAmount", "SalesAmount"));

// Appliquer les transformations au jeu complet
var pipelineModel = pipeline.Fit(fullDataView);
var preprocessedTrainData = pipelineModel.Transform(trainDataView);
var preprocessedTestData = pipelineModel.Transform(testDataView);

// Extraire les valeurs de Features et SalesAmount pour Infer.NET
var trainFeatures = mlContext.Data.CreateEnumerable<TransformedSalesData>(preprocessedTrainData, reuseRowObject: false).Select(x => x.Features).ToArray();

var trainLabels = mlContext.Data.CreateEnumerable<TransformedSalesData>(preprocessedTrainData, reuseRowObject: false).Select(x => (double)x.SalesAmount).ToArray();

var testFeatures = mlContext.Data.CreateEnumerable<TransformedSalesData>(preprocessedTestData, reuseRowObject: false).Select(x => x.Features).ToArray();

var testLabels = mlContext.Data.CreateEnumerable<TransformedSalesData>(preprocessedTestData, reuseRowObject: false).Select(x => (double)x.SalesAmount).ToArray();

// Modèle de régression bayésienne avec Infer.NET
int nFeatures = trainFeatures[0].Length; // Nombre de caractéristiques après one-hot encoding et normalisation
int nTrainSamples = trainFeatures.Length;

var featureRange = new Range(nFeatures);
var sampleRange = new Range(nTrainSamples);

var w = Variable.Array<double>(featureRange).Named("w");

// Initialiser w avec une distribution gaussienne
w[featureRange] = Variable<double>.Random(Gaussian.FromMeanAndVariance(0, 1)).ForEach(featureRange);

var x = Variable.Array(Variable.Array<double>(featureRange), sampleRange).Named("x");
var y = Variable.Array<double>(sampleRange).Named("y");

var noise = Variable.GaussianFromMeanAndVariance(0, 0.01).Named("noise");
using (Variable.ForEach(sampleRange))
{
    y[sampleRange] = Variable.GaussianFromMeanAndVariance(Variable.InnerProduct(w, x[sampleRange]), noise);
}

x.ObservedValue = trainFeatures.Select(t => t.Cast<double>().ToArray()).ToArray();
y.ObservedValue = trainLabels;

var ie = new InferenceEngine();
var wPosterior = ie.Infer<Gaussian[]>(w);

// Effectuer des prédictions sur le jeu de test
double[] Predict(double[][] features, Gaussian[] weights)
{
    var predictions = new double[features.Length];
    for (int i = 0; i < features.Length; i++)
    {
        double prediction = 0;
        for (int j = 0; j < features[i].Length; j++)
        {
            prediction += features[i][j] * weights[j].GetMean();
        }
        predictions[i] = prediction;
    }
    return predictions;
}

var trainPredictions = Predict(trainFeatures.Select(t => t.Cast<double>().ToArray()).ToArray(), wPosterior);
var testPredictions = Predict(testFeatures.Select(t => t.Cast<double>().ToArray()).ToArray(), wPosterior);

// Calculer les métriques de performance pour les ensembles d'entraînement et de test
double MeanAbsoluteError(double[] actual, double[] predicted)
{
    double sum = 0;
    for (int i = 0; i < actual.Length; i++)
    {
        sum += Math.Abs(actual[i] - predicted[i]);
    }
    return sum / actual.Length;
}

double trainMAE = MeanAbsoluteError(trainLabels, trainPredictions);
double testMAE = MeanAbsoluteError(testLabels, testPredictions);

Console.WriteLine("Performance sur le jeu d'entraînement :");
Console.WriteLine($"  Mean Absolute Error : {trainMAE}");

Console.WriteLine("Performance sur le jeu de test :");
Console.WriteLine($"  Mean Absolute Error : {testMAE}");

var trainAges = mlContext.Data.CreateEnumerable<SalesData>(trainDataView, reuseRowObject: false).Select(x => (double)x.Age).ToArray();
var trainIncomes = mlContext.Data.CreateEnumerable<SalesData>(trainDataView, reuseRowObject: false).Select(x => (double)x.Income).ToArray();

var testAges = mlContext.Data.CreateEnumerable<SalesData>(testDataView, reuseRowObject: false).Select(x => (double)x.Age).ToArray();
var testIncomes = mlContext.Data.CreateEnumerable<SalesData>(testDataView, reuseRowObject: false).Select(x => (double)x.Income).ToArray();

// Tracer les données d'entraînement réelles et prédites en 3D
var trainTrace1 = new Scatter3d
{
    x = trainAges,
    y = trainIncomes,
    z = trainLabels,
    mode = "lines",
    name = "Train Actual Sales"
};

var trainTrace2 = new Scatter3d
{
    x = trainAges,
    y = trainIncomes,
    z = trainPredictions,
    mode = "lines",
    name = "Train Predicted Sales"
};

// Tracer les données de test réelles et prédites en 3D
var testTrace1 = new Scatter3d
{
    x = testAges,
    y = testIncomes,
    z = testLabels,
    mode = "lines",
    name = "Test Actual Sales"
};

var testTrace2 = new Scatter3d
{
    x = testAges,
    y = testIncomes,
    z = testPredictions,
    mode = "lines",
    name = "Test Predicted Sales"
};

var layout = new Layout.Layout
{
    title = "Comparaison des ventes réelles et prédites en 3D",
    scene = new Scene
    {
        xaxis = new Xaxis
        {
            title = "Âge"
        },
        yaxis = new Yaxis
        {
            title = "Revenu"
        },
        zaxis = new Zaxis
        {
            title = "Montant des Ventes"
        }
    }
};

var chart = Chart.Plot(new[] { trainTrace1, trainTrace2, testTrace1, testTrace2 }, layout);
chart.Show();
